In [1]:
from lottery import Lottery

In [2]:
superenalotto = Lottery(max_numbers=90, max_extra=90, len_numbers=6, len_extra=0)
superenalotto(backend='choice', many=1_000_000).draw

Estrazione del: 11/09/2023 19:49 
Numeri Estratti: 13 34 42 46 80 86


In [3]:
superenalotto._stop

422775

In [3]:
eurojack = Lottery(max_numbers=50, max_extra=12, len_numbers=5, len_extra=3)
eurojack(backend='sample', many=500_000).draw

Estrazione del: 11/09/2023 19:54 
Numeri Estratti: 9 26 27 41 42
Superstar: 3 8 12


In [5]:
eurojack._stop

431551

In [6]:
winForLife = Lottery(20, 20, 10, 1)
winForLife(backend='sample', many=500_000).draw

Estrazione del: 06/09/2023 17:37 
Numeri Estratti: 1 4 8 10 11 12 13 14 19 20
Superstar: 15


In [7]:
winForLife._stop

28659

In [10]:
millionDay = Lottery(55, 0, 5, 0)
millionDay(backend='randint', many=500_000).draw

Estrazione del: 06/09/2023 17:38 
Numeri Estratti: 10 23 28 42 45


In [11]:
millionDay._stop

158937

In [55]:
superenalotto(backend='randint').draw

Estrazione del: 25/06/2023 10:22 
Numeri Estratti: 2 32 57 58 80 82


In [ ]:
superenalotto.backend

NameError: name 'superenalotto' is not defined

In [22]:
superenalotto.__dict__

{'max_numbers': 90,
 'max_extra': 90,
 'len_numbers': 6,
 'len_extra': 1,
 '_many': 1000000,
 '_backend': <function Lottery.Lottery.sample(_len, max)>,
 '_stop': 844763,
 '_combo': frozenset({4, 52, 56, 60, 67, 69}),
 '_extra': frozenset({5})}

In [214]:
# for b in ['choice', 'int', 'sample']:
#    print('######')
#    eurojack(backend=b, many=0)
#    print('backend:', eurojack.backend)
#
# print('\n################# many ####################\n')
#
# for b in ['choice', 'int', 'sample']:
#    print('######')
#    eurojack(backend=b)
#    print('backend:', eurojack.backend)

In [2]:
win = Lottery(20, 10, 20, 1)

w_1 = frozenset({2, 3, 5, 7, 9, 11, 12, 14, 16, 18})
w_2 = frozenset({7})

In [3]:
i = 1
win()
x_1, x_2 = win.numbers, win.extra
while ((w_1 != x_1) and (not w_1.isdisjoint(x_1))) or (w_2 != x_2):
    win()
    x_1, x_2 = win.numbers, win.extra
    i += 1
    if i % 1_000_000 == 0:
        print(i)
print(i, '>>>', x_1, x_2)

1000000
2000000
3000000


KeyboardInterrupt: 